In [566]:
import pandas as pd
import numpy as np
import math
import operator

In [567]:
df = pd.read_csv('/content/Suicide_Detection.csv')
df.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [568]:
dataframe=df.rename(columns={"class": "situation"})
inputs = dataframe.drop(['Unnamed: 0'],axis=1,inplace=True)
dummies = pd.get_dummies(dataframe.situation)
dummies.head(3)
dataframe.head()

,text,situation
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [569]:
 dataframe= pd.concat([dataframe,dummies],axis='columns')
dataframe.head(3)

,text,situation,non-suicide,suicide
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,0,1
1,Am I weird I don't get affected by compliments...,non-suicide,1,0
2,Finally 2020 is almost over... So I can never ...,non-suicide,1,0


In [570]:
inputs=dataframe.drop(['situation','non-suicide'],axis=1,inplace=True)
target = dataframe.suicide
dataframe.head()

,text,suicide
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,"I’m so lostHello, my name is Adam (16) and I’v...",1


In [571]:
dataframe.columns = dataframe.columns.str.replace(r"[^a-zA-Z\d\_]+", "")
dataframe = dataframe.replace([";",":","=","\+","<", ">", "\?", "!", "\\\\", "@", "#", "$", "\*", "%",
                               ",", "\.", "\(", "\)", "\[", "\]", "\{", "\}", "\"", "/br"], " ", regex = True)
dataframe = dataframe.replace(["' ", " '"], "", regex = True)
dataframe.head()

,text,suicide
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,I’m so lostHello my name is Adam 16 and I’v...,1


In [572]:
df_train = pd.DataFrame()
df_dev = pd.DataFrame()
df_test = pd.DataFrame()
split_1 = int(0.1 * len(dataframe))
split_2 = int(0.5 * len(dataframe))
df_train = dataframe[:split_1]
df_dev = dataframe[split_1:split_2]
df_test = dataframe[split_2:]

In [573]:
df_train_len = len(df_train)

df_train_pos_len = len(df_train[df_train['suicide'] == 0])
df_train_neg_len = len(df_train[df_train['suicide'] == 1])
prob_pos_train = df_train_pos_len / df_train_len
prob_neg_train = df_train_neg_len / df_train_len

In [574]:
wordfreq = dict()
wordfreq_pos = dict()
wordfreq_neg = dict()
for ind in df_train.index:
    review_set = set(df_train['text'][ind].lower().split())
    for word in review_set:
        if word in wordfreq:
            wordfreq[word] += 1
        else:
            wordfreq[word] = 1
        
        if df_train['suicide'][ind] == 0:
            if word in wordfreq_pos:
                wordfreq_pos[word] += 1
            else:
                wordfreq_pos[word] = 1
        else:
            if word in wordfreq_neg:
                wordfreq_neg[word] += 1
            else:
                wordfreq_neg[word] = 1

In [575]:
final_vocab = dict()
final_vocab_pos = dict()
final_vocab_neg = dict()
for word in wordfreq:
    if wordfreq[word] > 850:
        final_vocab[word] = wordfreq[word]
    if word in wordfreq_pos:
        if wordfreq_pos[word] > 850:
            final_vocab_pos[word] = wordfreq_pos[word]
    if word in wordfreq_neg:
        if wordfreq_neg[word] > 850:
            final_vocab_neg[word] = wordfreq_neg[word]

In [576]:
prob_word = dict()
prob_word_g_pos = dict()
prob_word_g_neg = dict()

for word in final_vocab:
    prob_word[word] = final_vocab[word] / df_train_len
    if word in final_vocab_pos:
        prob_word_g_pos[word] = final_vocab_pos[word] / df_train_pos_len
        
    if word in final_vocab_neg:
        prob_word_g_neg[word] = final_vocab_neg[word] / df_train_neg_len

In [577]:
accuracy_normal = []
df_dev_arr = np.array_split(df_dev, 5)
ctr = 0

print("Accuracy using 5-Fold Cross Validation:")

for df in df_dev_arr:
    count = 0
    ctr += 1
    predicted_sentiments = []
    prob_pos_g_wir = dict()
    prob_neg_g_wir = dict()
    
    for ind in df.index:
        numPos = 0.00
        numNeg = 0.00
        
        review_set = set(df['text'][ind].lower().split())
        for word in review_set:
            if word in prob_word:
                if word not in prob_word_g_pos:
                    numNeg = 0
                elif word not in prob_word_g_neg:
                    numPos = 0
                else:
                    numPos = numPos + math.log(prob_word_g_pos[word])
                    numNeg = numNeg + math.log(prob_word_g_neg[word])
                            
        prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
        prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
                            
        if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
            predicted_sentiments.append(0)
        else:
            predicted_sentiments.append(1)
                                    
    df['prediction'] = predicted_sentiments
                                                                        
    for ind in df.index:
        if df['suicide'][ind] == df['prediction'][ind]:
            count += 1
                                            
    accuracy = count / len(df)
    accuracy_normal.append(accuracy)
    print (ctr,": Accuracy df_dev:",accuracy*100,"%")

Accuracy using 5-Fold Cross Validation:
1 : Accuracy df_dev: 49.41290531078315 %
2 : Accuracy df_dev: 49.542173866207044 %
3 : Accuracy df_dev: 49.20823009802866 %
4 : Accuracy df_dev: 49.881503824194766 %
5 : Accuracy df_dev: 48.77194872347302 %


In [578]:
prob_word_g_pos_smooth = dict()
prob_word_g_neg_smooth = dict()

for word in final_vocab:
    if word in final_vocab_pos:
        prob_word_g_pos_smooth[word] = (final_vocab_pos[word]+1) / (df_train_pos_len + len(final_vocab))
        
    if word in final_vocab_neg:
        prob_word_g_neg_smooth[word] = (final_vocab_neg[word]+1) / (df_train_neg_len + len(final_vocab))

In [579]:
accuracy_smooth = []
df_dev_arr = np.array_split(df_dev, 5)
ctr = 0

print("Accuracy after Smoothing using 5-Fold Cross Validation:")

for df in df_dev_arr:
    count = 0
    ctr += 1
    predicted_sentiments = []
    prob_pos_g_wir = dict()
    prob_neg_g_wir = dict()
    
    for ind in df.index:
        numPos = 0.00
        numNeg = 0.00
        
        review_set = set(df['text'][ind].lower().split())
        for word in review_set:
            if word in prob_word:
                if word not in prob_word_g_pos:
                    numNeg = 0
                elif word not in prob_word_g_neg:
                    numPos = 0
                else:
                    numPos = numPos + math.log(prob_word_g_pos_smooth[word])
                    numNeg = numNeg + math.log(prob_word_g_neg_smooth[word])
                            
        prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
        prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
                            
        if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
            predicted_sentiments.append(0)
        else:
            predicted_sentiments.append(1)
                                    
    df['prediction'] = predicted_sentiments
                                                                        
    for ind in df.index:
        if df['suicide'][ind] == df['prediction'][ind]:
            count += 1
                                            
    accuracy = count / len(df)
    accuracy_smooth.append(accuracy)
    print (ctr,": Accuracy df_dev:",accuracy*100,"%")

Accuracy after Smoothing using 5-Fold Cross Validation:
1 : Accuracy df_dev: 49.41290531078315 %
2 : Accuracy df_dev: 49.542173866207044 %
3 : Accuracy df_dev: 49.20823009802866 %
4 : Accuracy df_dev: 49.881503824194766 %
5 : Accuracy df_dev: 48.77194872347302 %


In [580]:
betterNormal = 0
betterSmoothing = 0

for i in range(len(accuracy_normal)):
    if accuracy_normal[i] > accuracy_smooth[i]:
        betterNormal += 1
    else:
        betterSmoothing +=1

if(betterNormal > betterSmoothing):
    print("For the given dev dataset, accuracy is better without smoothing")
else:
    print("For the given dev dataset, accuracy is better with smoothing")

For the given dev dataset, accuracy is better with smoothing


In [581]:
prob_pos_given_word = dict()
prob_neg_given_word = dict()

for word in final_vocab:
    if word in final_vocab_pos:
        prob_pos_given_word[word] = (prob_word_g_pos[word] * prob_pos_train) / prob_word[word]
    if word in final_vocab_neg:
        prob_neg_given_word[word] = (prob_word_g_neg[word] * prob_neg_train) / prob_word[word]

In [582]:
print("Top 10 words predicting positive class:")
prob_pos_given_word = sorted(prob_pos_given_word.items(), key=operator.itemgetter(1), reverse=True)


prob_pos_given_word[:10]

Top 10 words predicting positive class:


[('your', 0.5429149797570849),
 ('you', 0.5283708686750511),
 ('we', 0.45490584737363726),
 ('i', 0.4213099779700177),
 ('a', 0.4168080386271695),
 ('i’m', 0.4142235123367199),
 ('and', 0.3934108527131782),
 ('the', 0.38907096515283257),
 ('are', 0.38364424345204445),
 ('to', 0.3834221155534742)]

In [583]:
print("Top 10 words predicting negative class:")
prob_neg_given_word = sorted(prob_neg_given_word.items(), key=operator.itemgetter(1), reverse=True)

prob_neg_given_word[:10]

Top 10 words predicting negative class:


[('suicidal', 0.9751908396946563),
 ('suicide', 0.9689922480620154),
 ('killing', 0.9628796400449943),
 ('kill', 0.945713216023961),
 ('depression', 0.9435028248587569),
 ('pain', 0.9296730413325106),
 ('die', 0.9272986457590876),
 ('living', 0.9240271772699197),
 ('thoughts', 0.922919508867667),
 ('job', 0.9225526641883519)]

In [584]:
predicted_sentiments = []
prob_pos_g_wir = dict()
prob_neg_g_wir = dict()

if(betterNormal < betterSmoothing):
    prob_word_g_pos = prob_word_g_pos_smooth

for ind in df_test.index:
    numPos = 0.00
    numNeg = 0.00
    
    review_set = set(df_test['text'][ind].lower().split())
    for word in review_set:
        if word in prob_word:
            if word not in prob_word_g_pos:
                numNeg = 0
            elif word not in prob_word_g_neg:
                numPos = 0
            else:
                numPos = numPos + math.log(prob_word_g_pos[word])
                numNeg = numNeg + math.log(prob_word_g_neg[word])
    
    prob_pos_g_wir[ind] = pow(math.e, numPos) * prob_pos_train
    prob_neg_g_wir[ind] = pow(math.e, numNeg) * prob_neg_train
    
    if(prob_pos_g_wir[ind] < prob_neg_g_wir[ind]):
        predicted_sentiments.append(0)
    else:
        predicted_sentiments.append(1)
        
df_test['prediction'] = predicted_sentiments

count = 0

for ind in df_test.index:
    if df_test['suicide'][ind] == df_test['prediction'][ind]:
        count += 1
        
accuracy = count / len(df_test)
print ("Accuracy df_test:",accuracy*100,"%")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Accuracy df_test: 49.621241500555854 %


In [585]:
#input_review = input('Enter your text: ') 
#abc1=0
#abc2=0
#for s1 in input_review.index:
 #   review_set = set(input_review['text'][s1].lower().split())
  #  for word in final_vocab_pos:
   #     abc1=(final_vocab_pos[word]+1) / (df_train_pos_len + len(final_vocab))
    #for word in final_vocab_neg:
     #   abc2=(final_vocab_neg[word]+1) / (df_train_neg_len + len(final_vocab))
#if abc1>abc2:
 # print ("Person is not Suicidal")
#else:
 # print ("Person is Suicidal")